In [1568]:
import numpy as np 
import random
import matplotlib.pyplot as plt # Graphical library
#from sklearn.metrics import mean_squared_error # Mean-squared error function

# Coursework 1 :
See pdf for instructions. 

In [1569]:
# WARNING: fill in these two functions that will be used by the auto-marking script
# [Action required]

def get_CID():
  return "01701146" # Return your CID (add 0 at the beginning to ensure it is 8 digits long)

def get_login():
  return "na2919" # Return your short imperial login

## Helper class

In [1570]:
# This class is used ONLY for graphics
# YOU DO NOT NEED to understand it to work on this coursework

class GraphicsMaze(object):

  def __init__(self, shape, locations, default_reward, obstacle_locs, absorbing_locs, absorbing_rewards, absorbing):

    self.shape = shape
    self.locations = locations
    self.absorbing = absorbing

    # Walls
    self.walls = np.zeros(self.shape)
    for ob in obstacle_locs:
      self.walls[ob] = 20

    # Rewards
    self.rewarders = np.ones(self.shape) * default_reward
    for i, rew in enumerate(absorbing_locs):
      self.rewarders[rew] = 10 if absorbing_rewards[i] > 0 else -10

    # Print the map to show it
    self.paint_maps()

  def paint_maps(self):
    """
    Print the Maze topology (obstacles, absorbing states and rewards)
    input: /
    output: /
    """
    plt.figure(figsize=(15,10))
    plt.imshow(self.walls + self.rewarders)
    plt.show()

  def paint_state(self, state):
    """
    Print one state on the Maze topology (obstacles, absorbing states and rewards)
    input: /
    output: /
    """
    states = np.zeros(self.shape)
    states[state] = 30
    plt.figure(figsize=(15,10))
    plt.imshow(self.walls + self.rewarders + states)
    plt.show()

  def draw_deterministic_policy(self, Policy):
    """
    Draw a deterministic policy
    input: Policy {np.array} -- policy to draw (should be an array of values between 0 and 3 (actions))
    output: /
    """
    plt.figure(figsize=(15,10))
    plt.imshow(self.walls + self.rewarders) # Create the graph of the Maze
    for state, action in enumerate(Policy):
      if(self.absorbing[0,state]): # If it is an absorbing state, don't plot any action
        continue
      arrows = [r"$\uparrow$",r"$\rightarrow$", r"$\downarrow$", r"$\leftarrow$"] # List of arrows corresponding to each possible action
      action_arrow = arrows[action] # Take the corresponding action
      location = self.locations[state] # Compute its location on graph
      plt.text(location[1], location[0], action_arrow, ha='center', va='center') # Place it on graph
    plt.show()

  def draw_policy(self, Policy):
    """
    Draw a policy (draw an arrow in the most probable direction)
    input: Policy {np.array} -- policy to draw as probability
    output: /
    """
    deterministic_policy = np.array([np.argmax(Policy[row,:]) for row in range(Policy.shape[0])])
    self.draw_deterministic_policy(deterministic_policy)

  def draw_value(self, Value):
    """
    Draw a policy value
    input: Value {np.array} -- policy values to draw
    output: /
    """
    plt.figure(figsize=(15,10))
    plt.imshow(self.walls + self.rewarders) # Create the graph of the Maze
    for state, value in enumerate(Value):
      if(self.absorbing[0, state]): # If it is an absorbing state, don't plot any value
        continue
      location = self.locations[state] # Compute the value location on graph
      plt.text(location[1], location[0], round(value,2), ha='center', va='center') # Place it on graph
    plt.show()

  def draw_deterministic_policy_grid(self, Policies, title, n_columns, n_lines):
    """
    Draw a grid representing multiple deterministic policies
    input: Policies {np.array of np.array} -- array of policies to draw (each should be an array of values between 0 and 3 (actions))
    output: /
    """
    plt.figure(figsize=(20,8))
    for subplot in range (len(Policies)): # Go through all policies
      ax = plt.subplot(n_columns, n_lines, subplot+1) # Create a subplot for each policy
      ax.imshow(self.walls+self.rewarders) # Create the graph of the Maze
      for state, action in enumerate(Policies[subplot]):
        if(self.absorbing[0,state]): # If it is an absorbing state, don't plot any action
          continue
        arrows = [r"$\uparrow$",r"$\rightarrow$", r"$\downarrow$", r"$\leftarrow$"] # List of arrows corresponding to each possible action
        action_arrow = arrows[action] # Take the corresponding action
        location = self.locations[state] # Compute its location on graph
        plt.text(location[1], location[0], action_arrow, ha='center', va='center') # Place it on graph
      ax.title.set_text(title[subplot]) # Set the title for the graph given as argument
    plt.show()

  def draw_policy_grid(self, Policies, title, n_columns, n_lines):
    """
    Draw a grid representing multiple policies (draw an arrow in the most probable direction)
    input: Policy {np.array} -- array of policies to draw as probability
    output: /
    """
    deterministic_policies = np.array([[np.argmax(Policy[row,:]) for row in range(Policy.shape[0])] for Policy in Policies])
    self.draw_deterministic_policy_grid(deterministic_policies, title, n_columns, n_lines)

  def draw_value_grid(self, Values, title, n_columns, n_lines):
    """
    Draw a grid representing multiple policy values
    input: Values {np.array of np.array} -- array of policy values to draw
    output: /
    """
    plt.figure(figsize=(20,8))
    for subplot in range (len(Values)): # Go through all values
      ax = plt.subplot(n_columns, n_lines, subplot+1) # Create a subplot for each value
      ax.imshow(self.walls+self.rewarders) # Create the graph of the Maze
      for state, value in enumerate(Values[subplot]):
        if(self.absorbing[0,state]): # If it is an absorbing state, don't plot any value
          continue
        location = self.locations[state] # Compute the value location on graph
        plt.text(location[1], location[0], round(value,1), ha='center', va='center') # Place it on graph
      ax.title.set_text(title[subplot]) # Set the title for the graoh given as argument
    plt.show()


## Maze class

In [1571]:
# This class define the Maze environment

class Maze(object):

  # [Action required]
  def __init__(self):
    """
    Maze initialisation.
    input: /
    output: /
    """
    
    # [Action required]
    # Properties set from the CID
    self.y = 9
    self.z = 9
    self._prob_success = 0.8 + 0.02 * (9 - self.y) # float
    self._gamma = 0.8 + 0.02 * self.y # float
    self._goal = int(self.z % 4) # integer (0 for R0, 1 for R1, 2 for R2, 3 for R3)

    # Build the maze
    self._build_maze()
                              

  # Functions used to build the Maze environment 
  # You DO NOT NEED to modify them
  def _build_maze(self):
    """
    Maze initialisation.
    input: /
    output: /
    """

    # Properties of the maze
    self._shape = (13, 10)
    self._obstacle_locs = [
                          (1,0), (1,1), (1,2), (1,3), (1,4), (1,7), (1,8), (1,9), \
                          (2,1), (2,2), (2,3), (2,7), \
                          (3,1), (3,2), (3,3), (3,7), \
                          (4,1), (4,7), \
                          (5,1), (5,7), \
                          (6,5), (6,6), (6,7), \
                          (8,0), \
                          (9,0), (9,1), (9,2), (9,6), (9,7), (9,8), (9,9), \
                          (10,0)
                         ] # Location of obstacles
    self._absorbing_locs = [(2,0), (2,9), (10,1), (12,9)] # Location of absorbing states
    self._absorbing_rewards = [ (500 if (i == self._goal) else -50) for i in range (4) ]
    self._starting_locs = [(0,0), (0,1), (0,2), (0,3), (0,4), (0,5), (0,6), (0,7), (0,8), (0,9)] #Reward of absorbing states
    self._default_reward = -1 # Reward for each action performs in the environment
    self._max_t = 500 # Max number of steps in the environment

    # Actions
    self._action_size = 4
    self._direction_names = ['N','E','S','W'] # Direction 0 is 'N', 1 is 'E' and so on
        
    # States
    self._locations = []
    for i in range (self._shape[0]):
      for j in range (self._shape[1]):
        loc = (i,j) 
        # Adding the state to locations if it is no obstacle
        if self._is_location(loc):
          self._locations.append(loc)
    self._state_size = len(self._locations)

    # Neighbours - each line is a state, ranked by state-number, each column is a direction (N, E, S, W)
    self._neighbours = np.zeros((self._state_size, 4)) 
    
    for state in range(self._state_size):
      loc = self._get_loc_from_state(state)

      # North
      neighbour = (loc[0]-1, loc[1]) # North neighbours location
      if self._is_location(neighbour):
        self._neighbours[state][self._direction_names.index('N')] = self._get_state_from_loc(neighbour)
      else: # If there is no neighbour in this direction, coming back to current state
        self._neighbours[state][self._direction_names.index('N')] = state

      # East
      neighbour = (loc[0], loc[1]+1) # East neighbours location
      if self._is_location(neighbour):
        self._neighbours[state][self._direction_names.index('E')] = self._get_state_from_loc(neighbour)
      else: # If there is no neighbour in this direction, coming back to current state
        self._neighbours[state][self._direction_names.index('E')] = state

      # South
      neighbour = (loc[0]+1, loc[1]) # South neighbours location
      if self._is_location(neighbour):
        self._neighbours[state][self._direction_names.index('S')] = self._get_state_from_loc(neighbour)
      else: # If there is no neighbour in this direction, coming back to current state
        self._neighbours[state][self._direction_names.index('S')] = state

      # West
      neighbour = (loc[0], loc[1]-1) # West neighbours location
      if self._is_location(neighbour):
        self._neighbours[state][self._direction_names.index('W')] = self._get_state_from_loc(neighbour)
      else: # If there is no neighbour in this direction, coming back to current state
        self._neighbours[state][self._direction_names.index('W')] = state

    # Absorbing
    self._absorbing = np.zeros((1, self._state_size))
    for a in self._absorbing_locs:
      absorbing_state = self._get_state_from_loc(a)
      self._absorbing[0, absorbing_state] = 1

    # Transition matrix
    self._T = np.zeros((self._state_size, self._state_size, self._action_size)) # Empty matrix of domension S*S*A
    for action in range(self._action_size):
      for outcome in range(4): # For each direction (N, E, S, W)
        # The agent has prob_success probability to go in the correct direction
        if action == outcome:
          prob = 1 - 3.0 * ((1.0 - self._prob_success) / 3.0) # (theoritically equal to self.prob_success but avoid rounding error and garanty a sum of 1)
        # Equal probability to go into one of the other directions
        else:
          prob = (1.0 - self._prob_success) / 3.0
          
        # Write this probability in the transition matrix
        for prior_state in range(self._state_size):
          # If absorbing state, probability of 0 to go to any other states
          if not self._absorbing[0, prior_state]:
            post_state = self._neighbours[prior_state, outcome] # Post state number
            post_state = int(post_state) # Transform in integer to avoid error
            self._T[prior_state, post_state, action] += prob

    # Reward matrix
    self._R = np.ones((self._state_size, self._state_size, self._action_size)) # Matrix filled with 1
    self._R = self._default_reward * self._R # Set default_reward everywhere
    for i in range(len(self._absorbing_rewards)): # Set absorbing states rewards
      post_state = self._get_state_from_loc(self._absorbing_locs[i])
      self._R[:,post_state,:] = self._absorbing_rewards[i]

    # Creating the graphical Maze world
    self._graphics = GraphicsMaze(self._shape, self._locations, self._default_reward, self._obstacle_locs, self._absorbing_locs, self._absorbing_rewards, self._absorbing)
    
    # Reset the environment
    self.reset()


  def _is_location(self, loc):
    """
    Is the location a valid state (not out of Maze and not an obstacle)
    input: loc {tuple} -- location of the state
    output: _ {bool} -- is the location a valid state
    """
    if (loc[0] < 0 or loc[1] < 0 or loc[0] > self._shape[0]-1 or loc[1] > self._shape[1]-1):
      return False
    elif (loc in self._obstacle_locs):
      return False
    else:
      return True


  def _get_state_from_loc(self, loc):
    """
    Get the state number corresponding to a given location
    input: loc {tuple} -- location of the state
    output: index {int} -- corresponding state number
    """
    return self._locations.index(tuple(loc))


  def _get_loc_from_state(self, state):
    """
    Get the state number corresponding to a given location
    input: index {int} -- state number
    output: loc {tuple} -- corresponding location
    """
    return self._locations[state]

  # Getter functions used only for DP agents
  # You DO NOT NEED to modify them
  def get_T(self):
    return self._T

  def get_R(self):
    return self._R

  def get_absorbing(self):
    return self._absorbing

  # Getter functions used for DP, MC and TD agents
  # You DO NOT NEED to modify them
  def get_graphics(self):
    return self._graphics

  def get_action_size(self):
    return self._action_size

  def get_state_size(self):
    return self._state_size

  def get_gamma(self):
    return self._gamma

  # Functions used to perform episodes in the Maze environment
  def reset(self):
    """
    Reset the environment state to one of the possible starting states
    input: /
    output: 
      - t {int} -- current timestep
      - state {int} -- current state of the envionment
      - reward {int} -- current reward
      - done {bool} -- True if reach a terminal state / 0 otherwise
    """
    self._t = 0
    self._state = self._get_state_from_loc(self._starting_locs[random.randrange(len(self._starting_locs))])
    self._reward = 0
    self._done = False
    return self._t, self._state, self._reward, self._done

  def step(self, action):
    """
    Perform an action in the environment
    input: action {int} -- action to perform
    output: 
      - t {int} -- current timestep
      - state {int} -- current state of the envionment
      - reward {int} -- current reward
      - done {bool} -- True if reach a terminal state / 0 otherwise
    """

    # If environment already finished, print an error
    if self._done or self._absorbing[0, self._state]:
      print("Please reset the environment")
      return self._t, self._state, self._reward, self._done

    # Drawing a random number used for probaility of next state
    probability_success = random.uniform(0,1)

    # Look for the first possible next states (so get a reachable state even if probability_success = 0)
    new_state = 0
    while self._T[self._state, new_state, action] == 0: 
      new_state += 1
    assert self._T[self._state, new_state, action] != 0, "Selected initial state should be probability 0, something might be wrong in the environment."

    # Find the first state for which probability of occurence matches the random value
    total_probability = self._T[self._state, new_state, action]
    while (total_probability < probability_success) and (new_state < self._state_size-1):
     new_state += 1
     total_probability += self._T[self._state, new_state, action]
    assert self._T[self._state, new_state, action] != 0, "Selected state should be probability 0, something might be wrong in the environment."
    
    # Setting new t, state, reward and done
    self._t += 1
    self._reward = self._R[self._state, new_state, action]
    self._done = self._absorbing[0, new_state] or self._t > self._max_t
    self._state = new_state
    return self._t, self._state, self._reward, self._done

## DP Agent

In [1572]:
# This class define the Dynamic Programing agent 

class DP_agent(object):

  # [Action required]
  # WARNING: make sure this function can be called by the auto-marking script
  def solve(self, env):
    """
    Solve a given Maze environment using Dynamic Programming
    input: env {Maze object} -- Maze to solve
    output: 
      - policy {np.array} -- Optimal policy found to solve the given Maze environment 
      - V {np.array} -- Corresponding value function 
    """
    
    # Initialisation (can be edited)
    policy = np.zeros((env.get_state_size(), env.get_action_size())) 
    V = np.zeros(env.get_state_size())
    threshold = 0.1
    delta = threshold
    T = env.get_T()
    R = env.get_R()
    gamma = env.get_gamma()

    #### 
    # Add your code here
    # WARNING: for this agent only, you are allowed to access env.get_T(), env.get_R() and env.get_absorbing()
    ####

    # Value iteration
    # Compute optimal value function
    epoch = 0
    while delta >= threshold:
      delta = 0
      delta_state = 0
      for state in range(env.get_state_size()):
        # If not absorbing state
        if not env.get_absorbing()[0, state]:
          # Store previous state value
          v = V[state]
          # Compute new state value
          Q = np.zeros(4)
          for next_state in range(env.get_state_size()):
            Q += T[state, next_state, :] * (R[state, next_state, :] + gamma * V[next_state])
          V[state] = np.max(Q)
          # Update delta
          delta = max(delta, abs(v - V[state]))
      epoch += 1
    print('Number of epochs: ', epoch)
    
    # Compute optimal policy
    for state in range(env.get_state_size()):
      # If not absorbing state
      if not env.get_absorbing()[0, state]:
        # Compute Q values
        Q = np.zeros(4)
        for next_state in range(env.get_state_size()):
          Q += T[state, next_state, :] * (R[state, next_state, :] + gamma * V[next_state])
        # Compute optimal policy
        policy[state, np.argmax(Q)] = 1

    return policy, V

## MC agent

In [1573]:
# # This class define the Monte-Carlo agent

# class MC_agent(object):
  
#   def learning_curve(self, env):
#     learning_rewards = []
#     for i in range(25):
#       print('Run ', i)
#       _, _, total_rewards = self.solve(env)
#       learning_rewards.append(total_rewards)
#     learning_rewards = np.array(learning_rewards)
#     mean_rewards = np.mean(learning_rewards, axis = 0)
#     std_rewards = np.std(learning_rewards, axis = 0)
#     # plot the mean and standard deviation of the rewards on the same graph
#     plt.figure(figsize=(15,10))
#     plt.plot(mean_rewards, label = 'Mean', color = 'b', linewidth = 2)
#     plt.fill_between(range(len(mean_rewards)), mean_rewards + std_rewards, mean_rewards - std_rewards, alpha = 0.2, color = 'skyblue')
#     plt.title('Learning Curve for Monte Carlo Agent across 25 simulations', fontsize = 14)
#     plt.xlabel('Episode Number', fontsize = 12)
#     plt.ylabel('Total Reward', fontsize = 12)
#     legend_elements = [plt.Line2D([0], [0], color='b', label='Mean Rewards', linewidth = 2),
#                       plt.Rectangle((0, 0), 1, 1, fc="skyblue", alpha=0.3, edgecolor='none', label='Standard Deviation')]
#     plt.legend(handles=legend_elements, fontsize = 10, loc = 'lower right')
#     plt.grid(alpha = 0.3)
#     plt.show()
#     print('Last mean reward', learning_rewards[:,-5:-1])

  
#   # [Action required]
#   # WARNING: make sure this function can be called by the auto-marking script
#   def solve(self, env):
#     """
#     Solve a given Maze environment using Monte Carlo learning
#     input: env {Maze object} -- Maze to solve
#     output: 
#       - policy {np.array} -- Optimal policy found to solve the given Maze environment 
#       - values {list of np.array} -- List of successive value functions for each episode 
#       - total_rewards {list of float} -- Corresponding list of successive total non-discounted sum of reward for each episode 
#     """

#     def epsilon_greedy(policy, epsilon):
#       """
#       Compute an epsilon-greedy policy from a given policy
#       input: policy {np.array} -- policy to convert
#       output: epsilon_greedy_policy {np.array} -- epsilon-greedy policy
#       """
#       epsilon_greedy_policy = np.zeros(policy.shape)
#       for state in range(policy.shape[0]):
#         max_arg = np.argmax(policy[state, :])
#         for action in range(policy.shape[1]):
#           epsilon_greedy_policy[state, action] = epsilon / policy.shape[1]
#         epsilon_greedy_policy[state, max_arg] = 1 - epsilon + epsilon / policy.shape[1]
#       return epsilon_greedy_policy

#     # Initialisation (can be edited)
#     epsilon = 0.8
#     Q = np.random.rand(env.get_state_size(), env.get_action_size()) 
#     V = np.zeros(env.get_state_size())
#     policy = epsilon_greedy(np.zeros((env.get_state_size(), env.get_action_size())), epsilon) # Epsilon greedy policy
#     values = [V]
#     total_rewards = []
#     gamma = env.get_gamma()
#     returns = [[[] for i in range(env.get_action_size())] for j in range(env.get_state_size())]

#     #### 
#     # Add your code here
#     # WARNING: this agent only has access to env.reset() and env.step()
#     # You should not use env.get_T(), env.get_R() or env.get_absorbing() to compute any value

#     # On-policy first-visit MC control (for epsilon-soft policies)
#     # Loop for each episode
#     for episode_count in range(5000):
#       # Reset environment
#       t, state, reward, done = env.reset()
#       total_reward = 0
#       # Generate episode
#       states = []
#       rewards = []
#       actions = []
#       step_count = 0
#       while not done and step_count < 500:
#         step_count += 1
#         # Choose action
#         action = np.random.choice(env.get_action_size(), p = policy[state, :])
#         # Perform action
#         t, next_state, reward, done = env.step(action)
#         # Store transition
#         states.append(state)
#         rewards.append(reward)
#         actions.append(action)
#         total_reward += reward
#         state = next_state
#       # Compute returns
#       G = 0
#       for t in reversed(range(0, len(states))):
#         G = gamma * G + rewards[t]
#         returns[states[t]][actions[t]].append(G)
#         # Update Q
#         Q[states[t], actions[t]] = np.mean(returns[states[t]][actions[t]])
#         # Update policy
#         best_action = np.argmax(Q[states[t], :])
#         for action in range(env.get_action_size()):
#           if action == best_action:
#             policy[states[t], action] = 1 - epsilon + epsilon / env.get_action_size()
#           else:
#             policy[states[t], action] = epsilon / env.get_action_size()
#       # Compute total reward
#       total_rewards.append(total_reward)
#       # Update values
#       V = np.max(Q, axis = 1)
#       values.append(V)
#       # Update epsilon
#       epsilon = epsilon * 0.9995
#       # print(episode_count, epsilon)
#     ####
    
#     return policy, values, total_rewards

In [1578]:
## my mc
from matplotlib.rcsetup import ValidateInStrings
# This class define the Monte-Carlo agent
# check variance for every visit
class MC_agent(object):

  # [Action required]
  # WARNING: make sure this function can be called by the auto-marking script
  def get_e_soft_greedy_policy(self,env,epsilon,Q,state,policy):
    best_action = np.argmax(Q[state,:])
    policy[state,:] = epsilon/env.get_action_size()
    policy[state,best_action] += 1-epsilon
    return policy

  def get_trace(self,env, policy,action_size):
    time_step, state, _, done = env.reset() #reward is 0
    trace = []
    while done!=1: 
      rnd=random.random()
      total_probability=0
      for a in range(action_size):
        total_probability+=policy[state,a]
        if total_probability>rnd:
          action=a
          break
      #action = np.random.choice(env.get_action_size(), p = policy[state, :])
      time_step, next_state, reward, done = env.step(action)
      trace.append((state, action, reward))
      state = next_state
    return trace

  def solve(self, env):
    """
    Solve a given Maze environment using Monte Carlo learning
    input: env {Maze object} -- Maze to solve
    output:
      - policy {np.array} -- Optimal policy found to solve the given Maze environment
      - values {list of np.array} -- List of successive value functions for each episode
      - total_rewards {list of float} -- Corresponding list of successive total non-discounted sum of reward for each episode
    """

    # Initialisation (can be edited)
    episodes_no = 10000
    epsilon = 0.8
    decay_factor = 0.9995
    Q = np.random.rand(env.get_state_size(), env.get_action_size())
    policy = np.zeros((env.get_state_size(), env.get_action_size()))
    V = np.zeros(env.get_state_size())
    total_rewards = []
    values = [V]
    state_size = env.get_state_size()
    action_size = env.get_action_size()
    Returns = [[[] for _ in range(action_size)] for _ in range(state_size)]
    for state in range(state_size):
      policy = self.get_e_soft_greedy_policy(env,epsilon,Q,state,policy)

    for episode in range(episodes_no):
      reward_sum = 0
      trace = self.get_trace(env,policy,action_size)
      G = 0
      for time_step in range(len(trace) - 1, 0, -1):
        state,action,reward = trace[time_step]
        reward_sum +=reward
        G = env.get_gamma()*G + reward
        #if not any((s == state and a == action) for s, a, _ in trace[:time_step]):
        Returns[state][action].append(G)
        Q[state,action] = sum(Returns[state][action])/len(Returns[state][action])
        policy = self.get_e_soft_greedy_policy(env,epsilon,Q,state,policy)
      #for state in range(state_size):
      #   V[state] = np.max(Q[state,:])
      V = np.max(Q,axis=1)
      values.append(V)
      total_rewards.append(reward_sum)
      #if epsilon>0.4 and episode%10==0:
      epsilon = epsilon*decay_factor

    policy = np.zeros((env.get_state_size(), env.get_action_size()))
    for state in range(env.get_state_size()):
      action = np.argmax(Q[state,:])
      policy[state, action] = 1
    


    return policy, values, total_rewards

  def plot_learning_curve(self,env,no_trials,no_episodes):
      rewards_list=[]
      for trial in range(1,no_trials+1):
        print("run "+ str(trial))
        _,_,trial_reward = self.solve(env)
        rewards_list.append(trial_reward)
      #episodes = [episode for episode in no_episodes]
      rewards_array = np.array(rewards_list)
      mean_rewards = np.mean(rewards_array,axis=0)
      std_rewards = np.std(rewards_array,axis=0)
      plt.plot(range(1,len(mean_rewards)+1),mean_rewards, label='Mean', color='blue')
      plt.fill_between(range(1,len(mean_rewards)+1), mean_rewards+std_rewards,mean_rewards-std_rewards,alpha=0.4, label='Standard Deviation',color='skyblue')

      # Add labels and a legend
      plt.title('Learning Curve for Monte Carlo Agent across 25 simulations', fontsize = 14)
      plt.xlabel('Episode number')
      plt.ylabel('Total reward')
      plt.grid()
      legend_elements = [plt.Line2D([0], [0], color='b', label='Mean Rewards'),
                      plt.Rectangle((0, 0), 1, 1, fc="skyblue", alpha=0.3, edgecolor='none', label='Standard Deviation')]
      plt.legend(handles=legend_elements, loc = 'lower right')

      # Show the plot
      plt.show()

In [1575]:
# from matplotlib.rcsetup import ValidateInStrings
# # This class define the Monte-Carlo agent
# # check variance for every visit
# class MC_agent(object):

#   # [Action required]
#   # WARNING: make sure this function can be called by the auto-marking script

#   def get_e_soft_greedy_policy(self,env,epsilon,policy):
#     soft_policy = np.zeros(policy.shape)
#     for state in range(policy.shape[0]):
#       best_action = np.argmax(policy[state,:])
#       for action in range(policy.shape[1]):
#         soft_policy[state,action] = epsilon/policy.shape[1]
#       soft_policy[state,best_action] += 1-epsilon
#     return soft_policy
  

#   def solve(self, env):
#     """
#     Solve a given Maze environment using Monte Carlo learning
#     input: env {Maze object} -- Maze to solve
#     output:
#       - policy {np.array} -- Optimal policy found to solve the given Maze environment
#       - values {list of np.array} -- List of successive value functions for each episode
#       - total_rewards {list of float} -- Corresponding list of successive total non-discounted sum of reward for each episode
#     """

#     # Initialisation (can be edited)
#     episodes_no = 5000
#     epsilon = 0.99
#     decay_factor = 0.9999
#     Q = np.random.rand(env.get_state_size(), env.get_action_size())
#     policy =self.get_e_soft_greedy_policy(env,epsilon,np.zeros((env.get_state_size(), env.get_action_size())))
#     V = np.zeros(env.get_state_size())
#     total_rewards = []
#     values = [V]
#     state_size = env.get_state_size()
#     action_size = env.get_action_size()
#     gamma=env.get_gamma()
#     Returns = [[[] for _ in range(action_size)] for _ in range(state_size)]
    
#     for episode in range(episodes_no):
#       # reset
#       reward_sum = 0
#       t, state, reward, done = env.reset()
#       # sample trace
#       trace_states = []
#       trace_rewards = []
#       trace_actions = []
#       G = 0
#       while done!=1:
#         best_action = np.random.choice(action_size, p = policy[state, :])
#         _, next_state, reward, done = env.step(best_action)
#         trace_states.append(state)
#         trace_rewards.append(reward)
#         trace_actions.append(best_action)
#         reward_sum+=reward
#         state=next_state
#       for time_step in reversed(range(0,len(trace_states))):
#         G = gamma*G + trace_rewards[time_step]
#         #if not any((s == state and a == action) for s, a, _ in trace[:time_step]):
#         Returns[trace_states[time_step]][trace_actions[time_step]].append(G)
#         Q[trace_states[time_step],trace_actions[time_step]] = np.mean(Returns[trace_states[time_step]][trace_actions[time_step]])
#         best_action = np.argmax(Q[trace_states[time_step], :])
#         for action in range(action_size):
#           if action==best_action:
#             policy[trace_states[time_step],action] = 1-epsilon+epsilon/action_size
#           else:
#             policy[trace_states[time_step],action] = epsilon/action_size

#       V = np.max(Q,axis=1)
#       values.append(V)
#       total_rewards.append(reward_sum)
#       epsilon = epsilon*decay_factor
    


#     return policy, values, total_rewards

#   def plot_learning_curve(self,env,no_trials,no_episodes):
#       rewards_list=[]
#       for trial in range(1,no_trials+1):
#         print("run "+ str(trial))
#         _,_,trial_reward = self.solve(env)
#         rewards_list.append(trial_reward)
#       #episodes = [episode for episode in no_episodes]
#       rewards_array = np.array(rewards_list)
#       mean_rewards = np.mean(rewards_array,axis=0)
#       std_rewards = np.std(rewards_array,axis=0)
#       plt.plot(range(1,len(mean_rewards)+1),mean_rewards, label='Mean', color='blue')
#       plt.fill_between(range(1,len(mean_rewards)+1), mean_rewards+std_rewards,mean_rewards-std_rewards,alpha=0.5)
#       plt.title("Mean and standard deviation of total reward across 25 trials")
#       legend_el = [plt.Line2D([0], [0], color='b', label='Mean'),
#                       plt.Rectangle((0, 0), 1, 1, fc="skyblue", alpha=0.5, edgecolor='none', label='Standard Deviation')]
#       # Add labels and a legend
#       plt.xlabel('Episode number')
#       plt.ylabel('Total reward')
#       plt.legend(handles=legend_el, loc = 'lower right')

#       # Show the plot
#       plt.show()

## TD agent

In [1576]:
# This class define the Temporal-Difference agent

class TD_agent(object):

  def learning_curve(self, env):
    plt.figure(figsize=(15,10))
    # Constant epsilon = 0.1
    epsilon = 0.3
    alphas = [0.01, 0.04, 0.07, 0.1, 0.4, 0.7]
    for alpha in alphas:
      rewards_per_alpha = []
      for i in range(50):
        _, _, total_rewards = self.graph_solve(env, epsilon, alpha)
        rewards_per_alpha.append(total_rewards)
      rewards_per_alpha = np.array(rewards_per_alpha)
      mean_rewards = np.mean(rewards_per_alpha, axis = 0)
      plt.plot(mean_rewards, label = '\u03B1 = ' + str(alpha))
      print('alpha', alpha, 'last mean reward', rewards_per_alpha[:,-1])
    plt.title('Learning Curve for TD Agent for varying \u03B1 values and constant \u03B5 = 0.1', fontsize = 16)
    plt.xlabel('Episode Number', fontsize = 14)
    plt.ylabel('Total Reward', fontsize = 14)
    plt.legend(fontsize = 12, loc = 'lower right')
    plt.grid(alpha = 0.3)
    plt.show()
    # Constant alpha = 0.05
    plt.figure(figsize=(15,10))
    epsilons = [0, 0.1, 0.3, 0.5, 0.7, 0.9, 1]
    alpha = 0.05
    for epsilon in epsilons:
      rewards_per_epsilon = []
      for i in range(50):
        _, _, total_rewards = self.graph_solve(env, epsilon, alpha)
        rewards_per_epsilon.append(total_rewards)
      rewards_per_epsilon = np.array(rewards_per_epsilon)
      mean_rewards = np.mean(rewards_per_epsilon, axis = 0)
      plt.plot(mean_rewards, label = '\u03B5 = ' + str(epsilon))
      print('epsilon', epsilon, 'last mean reward', rewards_per_epsilon[:,-1])
    plt.title('Learning Curve for TD Agent for varying \u03B5 values and constant \u03B1 = 0.05', fontsize = 16)
    plt.xlabel('Episode Number', fontsize = 14)
    plt.ylabel('Total Reward', fontsize = 14)
    plt.legend(fontsize = 12, loc = 'lower right')
    plt.grid(alpha = 0.3)
    plt.show()
    
  
  # [Action required]
  # WARNING: make sure this function can be called by the auto-marking script
  def solve(self, env):
    """
    Solve a given Maze environment using Temporal Difference learning
    input: env {Maze object} -- Maze to solve
    output: 
      - policy {np.array} -- Optimal policy found to solve the given Maze environment 
      - values {list of np.array} -- List of successive value functions for each episode 
      - total_rewards {list of float} -- Corresponding list of successive total non-discounted sum of reward for each episode 
    """

    # Initialisation (can be edited)
    Q = np.random.rand(env.get_state_size(), env.get_action_size()) 
    V = np.zeros(env.get_state_size())
    policy = np.zeros((env.get_state_size(), env.get_action_size())) 
    values = [V]
    total_rewards = []
    gamma = env.get_gamma()
    epsilon = 0.99

    #### 
    # Add your code here
    # WARNING: this agent only has access to env.reset() and env.step()
    # You should not use env.get_T(), env.get_R() or env.get_absorbing() to compute any value
    ####

    # Q-Learning
    for episode_count in range(5000):
      # Update alpha
      alpha = (episode_count + 1) ** (-0.51)
      # Reset environment
      t, state, reward, done = env.reset()
      total_reward = 0
      # Generate episode
      step_count = 0
      while not done and step_count < 500:
        step_count += 1
        # Make an epsilon-greedy policy from Q and Choose action
        best_action = np.argmax(Q[state, :])
        for action in range(env.get_action_size()):
          if action == best_action:
            policy[state, action] = 1 - epsilon + epsilon / env.get_action_size()
          else:
            policy[state, action] = epsilon / env.get_action_size()
        action = np.random.choice(env.get_action_size(), p = policy[state, :])
        # Perform action
        t, next_state, reward, done = env.step(action)
        # Update Q
        Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state, :]) - Q[state, action])
        # Update state and total reward
        state = next_state
        total_reward += reward
      # Compute total reward
      total_rewards.append(total_reward)
      # Update values
      V = np.max(Q, axis = 1)
      values.append(V)
      # Update epsilon
      epsilon = epsilon * (0.9999995 ** episode_count)
      
      print('episode', episode_count, 'epsilon', epsilon, 'alpha', alpha)
    # Compute optimal policy
    for state in range(env.get_state_size()):
      policy[state, np.argmax(Q[state, :])] = 1
    
    return policy, values, total_rewards
  

  def graph_solve(self, env, epsilon, alpha):
    """
    Solve a given Maze environment using Temporal Difference learning
    input: env {Maze object} -- Maze to solve
    output: 
      - policy {np.array} -- Optimal policy found to solve the given Maze environment 
      - values {list of np.array} -- List of successive value functions for each episode 
      - total_rewards {list of float} -- Corresponding list of successive total non-discounted sum of reward for each episode 
    """

    # Initialisation (can be edited)
    Q = np.random.rand(env.get_state_size(), env.get_action_size()) 
    V = np.zeros(env.get_state_size())
    policy = np.zeros((env.get_state_size(), env.get_action_size())) 
    values = [V]
    total_rewards = []
    gamma = env.get_gamma()

    #### 
    # Add your code here
    # WARNING: this agent only has access to env.reset() and env.step()
    # You should not use env.get_T(), env.get_R() or env.get_absorbing() to compute any value
    ####

    # Q-Learning
    for episode_count in range(200):
      # Reset environment
      t, state, reward, done = env.reset()
      total_reward = 0
      # Generate episode
      step_count = 0
      while not done and step_count < 500:
        step_count += 1
        # Make an epsilon-greedy policy from Q and Choose action
        best_action = np.argmax(Q[state, :])
        for action in range(env.get_action_size()):
          if action == best_action:
            policy[state, action] = 1 - epsilon + epsilon / env.get_action_size()
          else:
            policy[state, action] = epsilon / env.get_action_size()
        action = np.random.choice(env.get_action_size(), p = policy[state, :])
        # Perform action
        t, next_state, reward, done = env.step(action)
        # Update Q
        Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state, :]) - Q[state, action])
        # Update state and total reward
        state = next_state
        total_reward += reward
      # Compute total reward
      total_rewards.append(total_reward)
      # Update values
      V = np.max(Q, axis = 1)
      values.append(V)

    # Compute optimal policy
    for state in range(env.get_state_size()):
      policy[state, np.argmax(Q[state, :])] = 1
    
    return policy, values, total_rewards

## Example main

In [1579]:
# Example main (can be edited)

### Question 0: Defining the environment

print("Creating the Maze:\n")
# maze = Maze()

# ### Question 1: Dynamic programming
# dp_agent = DP_agent()
# dp_policy, dp_value = dp_agent.solve(maze)

# print("Results of the DP agent:\n")
# maze.get_graphics().draw_policy(dp_policy)
# maze.get_graphics().draw_value(dp_value)


### Question 2: Monte-Carlo learning

mc_agent = MC_agent()
mc_policy, mc_values, total_rewards = mc_agent.solve(maze)

print("Results of the MC agent:\n")
maze.get_graphics().draw_policy(mc_policy)
maze.get_graphics().draw_value(mc_values[-1])

# mc_agent.learning_curve(maze)


# ## Question 3: Temporal-Difference learning

# td_agent = TD_agent()
# td_policy, td_values, total_rewards = td_agent.solve(maze)

# print("Results of the TD agent:\n")
# maze.get_graphics().draw_policy(td_policy)
# maze.get_graphics().draw_value(td_values[-1])

# td_agent.learning_curve(maze)

# maze = Maze()
# mc_agent = MC_agent()
# mc_agent.plot_learning_curve(maze,25,5000)

Creating the Maze:



KeyboardInterrupt: 